### Pip install

In [1]:
!pip install datasets transformer_lens wandb plotly line_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 964.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 46.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.6/717.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ActivationStoreParallel import ActivationsStore
from sparse_transcoder import SparseTranscoder
from transcoder_training_parallel import train_transcoder_on_language_model_parallel
from transcoder_runner_parallel import language_model_transcoder_runner_parallel
from dataclasses import dataclass
import transformer_lens
import torch
import wandb
from typing import Optional
from test_config import test_cfg
from sparsify_feature_map import sparsity_transcoder

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: biggs (biggs-University College London (UCL)). Use `wandb login --relogin` to force relogin


True

### Query/Key transcoders

In [4]:
@dataclass
class UnifiedConfig():
    # Common settings
    model_name: str = "gpt2-small"
    hook_point: str = "blocks.10.hook_resid_pre"
    ln: str = 'blocks.10.ln1.hook_scale'
    hook_point_layer: int = 10
    layer: int = 10
    d_in: int = 768
    d_out: int = 768
    n_head: int = 12
    d_head: int = 64
    dataset_path: str = "Skylion007/openwebtext"
    is_dataset_tokenized: bool = False
    training: bool = True
    attn_scores_normed = True
    
    # SAE Parameters
    as_sae: bool = True
    expansion_factor: int = 12   # TODO: NOT being used??
    d_hidden: int = 2400
    b_dec_init_method: str = "mean"
    norming_decoder_during_training = True
    
    # Training Parameters
    lr: float = 1e-5
    reg_coefficient: float = 4e-6
    lr_scheduler_name: Optional[str] = None
    train_batch_size: int = 2048
    context_size: int = 256
    lr_warm_up_steps: int = 5000
    
    # Activation Store Parameters
    n_batches_in_buffer: int = 128
    total_training_tokens: int = 10_000 * 1_000
    store_batch_size: int = 32
    use_cached_activations: bool = False
    
    # Resampling protocol
    feature_sampling_method: str = 'none'
    feature_sampling_window: int = 1000
    feature_reinit_scale: float = 0.2
    resample_batches: int = 1028
    dead_feature_window: int = 50000
    dead_feature_threshold: float = 1e-6
    
    # WANDB
    log_to_wandb: bool = True
    log_final_model_to_wandb: bool = False
    wandb_project: str = "sparsification"
    wandb_entity: Optional[str] = None
    wandb_log_frequency: int = 50
    entity: str = "biggs-University College London (UCL)"
    
    # Misc
    device: str = "cuda"
    eps: float = 1e-7
    seed: int = 42
    reshape_from_heads: bool = True
    n_checkpoints: int = 10
    checkpoint_path: str = "checkpoints"
    dtype: torch.dtype = torch.float32
    run_name: str = "qk_parallel"
    
    # Query-specific settings
    hook_transcoder_in_q: str = "blocks.10.hook_resid_pre"
    hook_transcoder_out_q: str = "blocks.10.attn.hook_q"
    target_q: str = "blocks.10.attn.hook_q"
    type_q: str = "resid_to_queries"
    
    # Key-specific settings
    hook_transcoder_in_k: str = "blocks.10.hook_resid_pre"
    hook_transcoder_out_k: str = "blocks.10.attn.hook_k"
    target_k: str = "blocks.10.attn.hook_k"
    type_k: str = "resid_to_keys"

cfg = UnifiedConfig()
cfg.run_name = f"{cfg.d_hidden}_{cfg.reg_coefficient}_{cfg.lr}"

In [ ]:
sparse_transcoder_Q, sparse_transcoder_K = language_model_transcoder_runner_parallel(cfg)

Running...
Loaded pretrained model gpt2-small into HookedTransformer
Dataset is not tokenized! Updating config.
Reinitializing b_dec with mean of activations
Previous distances: 27.712806701660156
New distances: 19.674325942993164
Reinitializing b_dec_out with mean of activations
Previous distances: 36.17184066772461
New distances: 28.02600860595703
Reinitializing b_dec with mean of activations
Previous distances: 27.712806701660156
New distances: 19.674325942993164
Reinitializing b_dec_out with mean of activations
Previous distances: 41.437225341796875
New distances: 31.178068161010742


details/frac_acc,▁▅▅▅▅▇▄▇▇▆▄▃▅▆▆▇▂▆▆▆▆▅▆▇▇▆▆▇▆▇▇█▅▅▇▆█▃▆▂
details/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
details/n_training_tokens,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
details/patt_max_diff,█▅▅▄▄▃▄▄▃▃▃▃▂▃▃▂▃▂▂▂▁▂▂▂▂▂▂▁▂▁▂▂▁▁▂▁▁▁▂▁
details/pred_key_mean,▁▃▄▄▆▆▆▆▇▇▇▇▇▇█▇▇▇▆▇▇██▇▇▇▇▇█▇▇▇█▆▇▇▇█▆▇
details/pred_query_mean,█▆█▆▆▄▄▄▄▃▃▃▂▃▃▃▃▃▁▃▂▄▄▂▃▃▃▂▃▃▂▂▄▃▃▃▃▄▃▄
losses/mse_lossK,█▅▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁
losses/mse_lossQ,█▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/patt_lossK,█▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▃▂▂▂▁▂▂▁▂▁▂▁▂▁▁▁▁▁▁▁▁▂▁▂
losses/patt_lossQ,█▄▄▃▃▃▃▃▂▂▃▂▂▂▂▂▃▂▂▂▁▂▂▁▂▁▂▁▂▁▁▁▁▁▁▁▁▂▁▂
losses/patt_loss_full,█▄▄▄▄▄▄▄▃▃▃▃▂▃▃▂▄▂▂▂▁▂▂▁▂▁▂▁▂▁▂▁▁▁▂▁▁▂▁▂


Initialising...


Training Transcoders:  10%|█         | 10020864/100000000 [04:00<1:33:58, 15958.83it/s, Loss=4.795]

Saved model to checkpoints/10000384_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/10000384_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt


...

Training Transcoders:  40%|████      | 40022016/100000000 [15:58<11:10, 89413.00it/s, Loss=1.035]  

Saved model to checkpoints/40001536_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/40001536_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt


Training Transcoders:  50%|█████     | 50024448/100000000 [19:55<09:39, 86237.97it/s, Loss=0.961]  

Saved model to checkpoints/50001920_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt
Saved model to checkpoints/50001920_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt


Training Transcoders:  51%|█████     | 50855936/100000000 [20:10<07:53, 103768.02it/s, Loss=0.922]

In [5]:
# train_transcoder_on_language_model_parallel(cfg, model, query_sae, key_sae, activations_store)

In [17]:
!cp checkpoints/final_sparse_transcoder_gpt2-small_resid_to_keys_2400.pt checkpoints/keys_sae_normed_final.pt
!cp checkpoints/final_sparse_transcoder_gpt2-small_resid_to_queries_2400.pt checkpoints/queries_sae_normed_final.pt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
query_sae = SparseTranscoder.load_from_pretrained("checkpoints/queries_sae_normed_final.pt", True)
key_sae = SparseTranscoder.load_from_pretrained("checkpoints/keys_sae_normed_final.pt", False)
cfg = query_sae.cfg
model = transformer_lens.HookedTransformer.from_pretrained(cfg.model_name, fold_ln=True)
activations_store = ActivationsStore(cfg, model)

Loaded pretrained model gpt2-small into HookedTransformer
Dataset is not tokenized! Updating config.


In [6]:
cfg.log_to_wandb = True
cfg.mask_reg_coeff = 4e-6
cfg.lr = 1e-3
mask = sparsity_transcoder(cfg, model, query_sae, key_sae, activations_store)

Training Mask:  16%|█▌        | 15728640/100000000 [09:36<51:29, 27280.36it/s, Patt Loss=0.125, Sparsity=32552.631, Fraction Zeros=1.000]    


KeyboardInterrupt: 

In [9]:
%config InteractiveShell

UsageError: Invalid config statement: 'InteractiveShell', should be `Class.trait = value`.
